In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import simfile
from simfile.notes import NoteData, NoteType
from simfile.timing import Beat, TimingData
from simfile.timing.engine import TimingEngine
import svg

In [ ]:
qtzn = ['#999999'] * 48

colors = {
    4: '#FF0000',
    8: '#0000FF',
    12: '#FF00FF',
    16: '#009900',
    24: '#9900FF',
    32: '#FF6600',
    48: '#CC0066',
    64: '#0099FF',
    96: '#66BBBB',
    192: '#336666'
}

for q in reversed([k for k in colors]):
    for i in range(0, 48, 192 // q):
        qtzn[i] = colors[q]

In [ ]:
def plot_rhythms(sm, chart = None, measure_length: float = 4, time_based: bool = False):
    if chart is None:
        chart = sm.charts[0]
    note_data = NoteData(chart)
    note_positions = []

    if time_based:
        engine = TimingEngine(TimingData(sm, chart))
        # print(TimingData(sm, chart).bpms)
        for note in note_data:
            if note.note_type not in [NoteType.MINE, NoteType.TAIL]:
                m = note.beat.numerator // (measure_length * note.beat.denominator)
                v = engine.time_at(note.beat) -- engine.time_at((measure_length * m))
                t = (note.beat.numerator * (48 // note.beat.denominator)) % 48
                # print(f'{note.beat}: {m} r{v:0.6f} ({t}) @ {engine.bpm_at(note.beat):0.3f} BPM')
                note_positions.append((m, v, t))
    else:
        for note in note_data:
            if note.note_type not in [NoteType.MINE, NoteType.TAIL]:
                m = note.beat.numerator // (measure_length * note.beat.denominator)
                v = note.beat -- (measure_length * m)
                t = (note.beat.numerator * (48 // note.beat.denominator)) % 48
                # print(f'{note.beat}: {m} r{v} ({t})')
                note_positions.append((m, v, t))
    
    fig_width = max([v[0] for v in note_positions])
    fig_height = 8 * measure_length
    fig_scale = 18

    fig = plt.figure(figsize=(fig_scale, fig_height * fig_scale / fig_width))
    plt.scatter(
        [v[0] for v in note_positions],
        [v[1] for v in note_positions],
        c = [qtzn[v[2]] for v in note_positions],
        marker = '.'
    )
    plt.show()

In [ ]:
simfile_test = simfile.open(
    r'C:\Games\ITGmania\Songs\RIP 13 Singles\[T10] Slaughta\slaughta.ssc'
)
plot_rhythms(simfile_test)


In [ ]:
simfile_test = simfile.open(
    r'C:\Games\ITGmania\Songs\RIP 13 Singles\[T10] DUAL BREAKER XX (No CMOD)\DUAL BREAKER XX.ssc'
)
plot_rhythms(simfile_test)

In [ ]:
simfile_test = simfile.open(
    r'C:\Games\ITGmania\Songs\RIP 13 Singles\[T10] Field of View\Field of View.ssc'
)
plot_rhythms(simfile_test)

In [ ]:
simfile_test = simfile.open(
    r'C:\Games\ITGmania\Songs\RIP 13 Singles\[T10] Arcology on Permafrost\Arcology On Permafrost.ssc'
)
plot_rhythms(simfile_test)

In [ ]:
simfile_test = simfile.open(
    r'C:\Games\ITGmania\Songs\RIP 13 Singles\[T10] bipolar\bipolar.ssc'
)
plot_rhythms(simfile_test)

In [ ]:
simfile_test = simfile.open(
    r'C:\Games\ITGmania\Songs\RIP 13 Singles\[T10] line theta\steps.ssc'
)
plot_rhythms(simfile_test)

In [ ]:
simfile_test = simfile.open(
    r'C:\Games\ITGmania\Songs\RIP 13 Singles\[T07] Quick Attack (No CMOD)\Quick Attack.sm'
)
plot_rhythms(simfile_test, time_based=True)
plot_rhythms(simfile_test, time_based=False)

In [ ]:
simfile_test = simfile.open(
    r'C:\Games\ITGmania\Songs\RIP 13 Singles\[T08] 7thSense (No CMOD)\7th.ssc'
)
plot_rhythms(simfile_test, time_based=True)
plot_rhythms(simfile_test, time_based=False)

### Guiding principles
perhaps these are radar points in an intermediate output?
#### Variety
the sheer quantity of different rhythmic patterns in a file regardless of where they appear
#### Novelty
the rate of switching between patterns the player is expected to do as they play through the song
#### Fill
the complexity of the patterns themselves given a particular subdivision
#### Subdivision
how the beat(s) are subdivided
### Need to measure
#### Difference between two patterns
I feel like there should be some sort of "dot product" for two pattern "vectors".
If they're unequal length maybe just truncate the longer one to use the later portion.
#### Difficulty of an isolated pattern
- subdivision matters: wholes < halves < quarters < eighths < sixths < sixteenths < twelfths < fifths < tenths < twentieths < sevenths < ...
- at some point the subdivision is so small that attempting to conceptualize the timing isn't worth it compared to rounding off, but the timing skill of the player changes this a LOT (think 10.0ms mean error player vs. 4.0ms mean error player)
- how to control flams and swing from being overweighted?
- is there a difference between x ---x and  ---xx? I think so
#### Selx--similarity?
- can we define a passage's rhythmic complexity in terms of partial passages?
#### Accounting for meter changes
- not always in 4/4, might be 2/4 or 3/4, or have an ofx--length measure thrown in


### Ranking (easiest to hardest)
- x-----------------------x
- x-----------x-----------x
- x-----x-----x-----x-----x
- x-----------x-----x-----x
- x-----x-----x-----------x
- x-----x-----------x-----x
- ------------x-----------x
- ------x-----x-----x-----x
- ------------x-----x-----x
- ------x-----x-----------x
- ------x-----------x-----x
- x-------x-------x-------x
- x-------x-------x---x---x
### I suspect that:
- Dividing in half > Original
- Trailing smallest quantization removal >= Leading smallest quantization removal > Fully articulated bar (e.g., 1e&. >= 1.&a > 1e&a)
- Downbeat removal > Offbeat removal > downbeat present
- 7-let rhythms > 5-let rhythms > Triplet rhythms > Duplet rhythms